In [37]:
# config file for ETL process

import pandas as pd
import sqlite3
from pathlib import Path

BASE_DIR = Path().resolve().parent
DB_PATH = BASE_DIR / "db" / "customer_dw.sqlite"
CSV_PATH = BASE_DIR / "data" / "new_retail_data.csv"


In [38]:
# Function to load raw CSV data

def load_raw_csv():
    df = pd.read_csv(CSV_PATH)
    return df

In [39]:
# Dimension Table: dim_customer
# Build dimension table for customers

def build_dim_customer(df):
    dim_customer = (
        df[
            [
                "Customer_ID",
                "Name",
                "Email",
                "Phone",
                "Address",
                "City",
                "State",
                "Zipcode",
                "Country",
                "Age",
                "Gender",
                "Income",
                "Customer_Segment",
            ]
        ]
        .drop_duplicates(subset=["Customer_ID"])
        .copy()
    )

    # Add surrogate key in Python
    dim_customer["customer_key"] = range(1, len(dim_customer) + 1)

    # Put key as first column
    cols = ["customer_key"] + [c for c in dim_customer.columns if c != "customer_key"]
    dim_customer = dim_customer[cols]

    print("[DONE] Dimension table 'dim_customer' built successfully.")
    return dim_customer


In [40]:
# Dimension Table: dim_product
# Build dimension table for products

def build_dim_product(df):
    dim_product = (
        df[["Product_Category", "Product_Brand", "Product_Type"]]
        .drop_duplicates()
        .copy()
    )

    dim_product = dim_product.rename(
        columns={
            "Product_Category": "Category",
            "Product_Brand": "Brand",
            "Product_Type": "Type",
        }
    )

    dim_product["product_key"] = range(1, len(dim_product) + 1)
    cols = ["product_key"] + [c for c in dim_product.columns if c != "product_key"]
    dim_product = dim_product[cols]

    print("[DONE] Dimension table 'dim_product' built successfully.")
    return dim_product


In [41]:
# Dimension Table: dim_date
# Build dimension table for dates

def build_dim_date(df):
    dates = pd.to_datetime(df["Date"], errors="coerce").dropna().drop_duplicates()
    dim_date = pd.DataFrame({"date_value": dates})

    dim_date["date_key"] = dim_date["date_value"].dt.strftime("%Y%m%d").astype(int)
    dim_date["day"] = dim_date["date_value"].dt.day
    dim_date["month"] = dim_date["date_value"].dt.month
    dim_date["month_name"] = dim_date["date_value"].dt.month_name()
    dim_date["quarter"] = dim_date["date_value"].dt.quarter
    dim_date["year"] = dim_date["date_value"].dt.year
    dim_date["day_of_week"] = dim_date["date_value"].dt.weekday  # 0=Mon
    dim_date["is_weekend"] = dim_date["day_of_week"].isin([5, 6]).astype(int)

    cols = [
        "date_key",
        "date_value",
        "day",
        "month",
        "month_name",
        "quarter",
        "year",
        "day_of_week",
        "is_weekend",
    ]
    dim_date = dim_date[cols].sort_values("date_value")

    print("[DONE] Dimension table 'dim_date' built successfully.")
    return dim_date


In [42]:
def build_dim_payment_method(df):
    dim_payment = (
        df[["Payment_Method"]].drop_duplicates().rename(columns={"Payment_Method": "payment_code"})
    )
    dim_payment["payment_method_key"] = range(1, len(dim_payment) + 1)
    cols = ["payment_method_key", "payment_code"]

    print("[DONE] Dimension table 'dim_payment_method' built successfully.")
    return dim_payment[cols]


def build_dim_shipping_method(df):
    dim_shipping = (
        df[["Shipping_Method"]].drop_duplicates().rename(columns={"Shipping_Method": "shipping_method"})
    )
    dim_shipping["shipping_method_key"] = range(1, len(dim_shipping) + 1)
    cols = ["shipping_method_key", "shipping_method"]

    print("[DONE] Dimension table 'dim_shipping_method' built successfully.")
    return dim_shipping[cols]


def build_dim_order_status(df):
    dim_status = (
        df[["Order_Status"]].drop_duplicates().rename(columns={"Order_Status": "order_status"})
    )
    dim_status["order_status_key"] = range(1, len(dim_status) + 1)
    cols = ["order_status_key", "order_status"]

    print("[DONE] Dimension table 'dim_order_status' built successfully.")
    return dim_status[cols]


In [46]:
def build_fact_sales(df,
                     dim_customer,
                     dim_product,
                     dim_date,
                     dim_payment,
                     dim_shipping,
                     dim_status):
    fact = df.copy()

    # --- 1) parse date for key ---
    fact["Date_parsed"] = pd.to_datetime(fact["Date"], errors="coerce")
    fact["date_key"] = fact["Date_parsed"].dt.strftime("%Y%m%d").astype(float)
    fact = fact.dropna(subset=["date_key"])
    fact["date_key"] = fact["date_key"].astype(int)

    # --- 2) make sure Total_Purchase_Amount exists ---
    if "Total_Purchase_Amount" not in fact.columns:
        fact["Total_Purchase_Amount"] = fact["Amount"] * fact["Total_Purchases"]

    # (optional) strip column names in case of trailing spaces
    fact.columns = [c.strip() for c in fact.columns]

    # --- 3) Join customer_key ---
    fact = fact.merge(
        dim_customer[["customer_key", "Customer_ID"]],
        left_on="Customer_ID",
        right_on="Customer_ID",
        how="left",
    )

    # --- 4) Join product_key ---
    fact = fact.merge(
        dim_product[["product_key", "Category", "Brand", "Type"]],
        left_on=["Product_Category", "Product_Brand", "Product_Type"],
        right_on=["Category", "Brand", "Type"],
        how="left",
    )

    # --- 5) Join payment_method_key ---
    fact = fact.merge(
        dim_payment,
        left_on="Payment_Method",
        right_on="payment_code",
        how="left",
    )

    # --- 6) Join shipping_method_key ---
    fact = fact.merge(
        dim_shipping,
        left_on="Shipping_Method",
        right_on="shipping_method",
        how="left",
    )

    # --- 7) Join order_status_key ---
    fact = fact.merge(
        dim_status,
        left_on="Order_Status",
        right_on="order_status",
        how="left",
    )

    # --- 8) Build final fact table ---
    fact_sales = fact[
        [
            "Transaction_ID",
            "customer_key",
            "product_key",
            "date_key",
            "payment_method_key",
            "shipping_method_key",
            "order_status_key",
            "Amount",
            "Total_Purchases",
            "Total_Purchase_Amount",
            "Ratings",
            "Feedback",
            "Time",
        ]
    ].rename(
        columns={
            "Transaction_ID": "transaction_id",
            "Amount": "amount",
            "Total_Purchases": "total_purchases",
            "Total_Purchase_Amount": "total_purchase_amount",
            "Ratings": "ratings",
            "Feedback": "feedback",
            "Time": "time",
        }
    )

    print("[DONE] Fact table 'fact_sales' built successfully.")
    return fact_sales


In [ ]:
def write_to_sqlite(dim_customer,
                    dim_product,
                    dim_date,
                    dim_payment,
                    dim_shipping,
                    dim_status,
                    fact_sales):
    DB_PATH.parent.mkdir(exist_ok=True, parents=True)
    conn = sqlite3.connect(DB_PATH)

    # Write dimension tables
    dim_customer.to_sql("dim_customer", conn, if_exists="replace", index=False)
    dim_product.to_sql("dim_product", conn, if_exists="replace", index=False)
    dim_date.to_sql("dim_date", conn, if_exists="replace", index=False)
    dim_payment.to_sql("dim_payment_method", conn, if_exists="replace", index=False)
    dim_shipping.to_sql("dim_shipping_method", conn, if_exists="replace", index=False)
    dim_status.to_sql("dim_order_status", conn, if_exists="replace", index=False)

    fact_sales.to_sql("fact_sales", conn, if_exists="replace", index=False)

    conn.close()
    print(f"[DONE] All tables written to SQLite database at: {DB_PATH}")


In [48]:
def main():
    df = load_raw_csv()

    dim_customer = build_dim_customer(df)
    dim_product = build_dim_product(df)
    dim_date = build_dim_date(df)
    dim_payment = build_dim_payment_method(df)
    dim_shipping = build_dim_shipping_method(df)
    dim_status = build_dim_order_status(df)

    fact_sales = build_fact_sales(
        df,
        dim_customer,
        dim_product,
        dim_date,
        dim_payment,
        dim_shipping,
        dim_status,
    )

    write_to_sqlite(
        dim_customer,
        dim_product,
        dim_date,
        dim_payment,
        dim_shipping,
        dim_status,
        fact_sales,
    )

if __name__ == "__main__":
    main()


[DONE] Dimension table 'dim_customer' built successfully.
[DONE] Dimension table 'dim_product' built successfully.
[DONE] Dimension table 'dim_date' built successfully.
[DONE] Dimension table 'dim_payment_method' built successfully.
[DONE] Dimension table 'dim_shipping_method' built successfully.
[DONE] Dimension table 'dim_order_status' built successfully.
[DONE] Fact table 'fact_sales' built successfully.
✅ All dimension and fact tables written to SQLite.
